In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.functions import year,date_format,to_date

In [21]:
Spark = SparkSession.builder.appName("Ecommerce_Analysis_using_SparkSQL").getOrCreate()

In [3]:
df = Spark.read.csv("./EcommerceFinal.csv",header = True,inferSchema = True)

In [4]:
df.createOrReplaceTempView("Ecommerce")

In [5]:
df = Spark.sql("SELECT * FROM Ecommerce")
df.show()

+--------------------+--------------------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|              Region|             Country|      Item Type|Sales Channel|Order Priority|Order Date| Order ID| Ship Date|Units Sold|Unit Price|Unit Cost|Total Revenue|Total Cost|Total Profit|
+--------------------+--------------------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|Central America a...| Antigua and Barbuda|      Baby Food|       Online|             M|2013-12-20|957081544|2014-01-11|       552|    255.28|   159.42|    140914.56|  87999.84|    52914.72|
|Central America a...|              Panama|         Snacks|      Offline|             C|2010-07-05|301644504|2010-07-26|      2167|    152.58|    97.44|    330640.86| 211152.48|   119488.38|
|              Europe|      Czech Republic|  

In [6]:
# Display the number of countries present in the table.
s = Spark.sql("SELECT Count(Distinct(Country)) as Total_Countries from Ecommerce")
s.show()

+---------------+
|Total_Countries|
+---------------+
|            185|
+---------------+



In [7]:
# Display the number of units sold in each region.
s2 = Spark.sql("SELECT Region,sum(`Units Sold`) as `Total_Units_Sold` FROM Ecommerce group by Region order by Total_Units_Sold desc")
s2.show()

+--------------------+----------------+
|              Region|Total_Units_Sold|
+--------------------+----------------+
|  Sub-Saharan Africa|         6642380|
|              Europe|         6582322|
|                Asia|         3620036|
|Middle East and N...|         3013431|
|Central America a...|         2698776|
|Australia and Oce...|         2111786|
|       North America|          484760|
+--------------------+----------------+



In [8]:
# Display the 10 most recent sales.
s3 = Spark.sql("SELECT * FROM Ecommerce order by `Order Date` desc")
s3.show(10)

+--------------------+--------------------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|              Region|             Country|      Item Type|Sales Channel|Order Priority|Order Date| Order ID| Ship Date|Units Sold|Unit Price|Unit Cost|Total Revenue|Total Cost|Total Profit|
+--------------------+--------------------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|                Asia|              Bhutan|         Cereal|      Offline|             M|2017-07-28|223854434|2017-08-25|      2356|     205.7|   117.11|     484629.2| 275911.16|   208718.04|
|Middle East and N...|United Arab Emirates|      Household|       Online|             C|2017-07-26|419542396|2017-08-08|       773|    668.27|   502.54|    516572.71| 388463.42|   128109.29|
|  Sub-Saharan Africa|             Senegal|  

In [9]:
# Display the products with at least 2 occurrences of ‘a’.
s4 = Spark.sql("SELECT `Item Type` FROM Ecommerce Where `Item Type` LIKE '%a%a%'")
# s4.show()

In [10]:
s4.groupBy("Item Type").agg(f.count("Item Type")).show()

+-------------+----------------+
|    Item Type|count(Item Type)|
+-------------+----------------+
|Personal Care|             415|
+-------------+----------------+



In [11]:
# Display country in each region with highest units sold.
Spark.sql("SELECT Region,Country,`Sales Channel`,max(`Units Sold`) as `Highest_Units_Sold` FROM Ecommerce group by Region,Country,`Sales Channel` order  by `Highest_Units_Sold` desc").show()

+--------------------+-------------------+-------------+------------------+
|              Region|            Country|Sales Channel|Highest_Units_Sold|
+--------------------+-------------------+-------------+------------------+
|              Europe|     United Kingdom|      Offline|              9999|
|              Europe|           Portugal|      Offline|              9997|
|Central America a...|           Dominica|       Online|              9996|
|              Europe|            Ireland|       Online|              9996|
|                Asia|           Thailand|       Online|              9991|
|Australia and Oce...|              Samoa|       Online|              9990|
|              Europe|         San Marino|      Offline|              9985|
|  Sub-Saharan Africa|  Equatorial Guinea|       Online|              9984|
|Central America a...|           Barbados|       Online|              9977|
|Central America a...|             Panama|       Online|              9976|
|  Sub-Sahar

In [13]:
# Display the unit price and unit cost of each item. #By Pyspark
# Spark.sql("SELECT Item Type,`Unit Price`,`Unit Cost` FROM Ecommerce group by Item Type").show()

In [14]:
# Display the number of sales yearwise

df = df.withColumn("Extr_Year",year("Order Date"))
# df.show()

df.groupby("Extr_Year").agg(f.count("Order ID").alias("No Of Sales")).sort("Extr_Year",ascending = False).show()

+---------+-----------+
|Extr_Year|No Of Sales|
+---------+-----------+
|     2017|        363|
|     2016|        670|
|     2015|        679|
|     2014|        660|
|     2013|        660|
|     2012|        678|
|     2011|        658|
|     2010|        632|
+---------+-----------+



In [15]:
# Display the number of orders for each item.
Spark.sql("SELECT `Item Type`,Count(`Order ID`) as `number of orders` From Ecommerce group by `Item Type`").show()

+---------------+----------------+
|      Item Type|number of orders|
+---------------+----------------+
|      Baby Food|             445|
|         Cereal|             385|
|           Meat|             399|
|      Household|             424|
|     Vegetables|             410|
|      Beverages|             447|
|Office Supplies|             420|
|      Cosmetics|             424|
|  Personal Care|             415|
|         Fruits|             447|
|         Snacks|             398|
|        Clothes|             386|
+---------------+----------------+



In [16]:
# Display the country with highest sale
Spark.sql("SELECT Country,sum(`Total Revenue`) as Total_sale_by_country FROM Ecommerce group by Country order by Total_sale_by_country desc").collect()

[Row(Country='Rwanda', Total_sale_by_country=60398739.58999999),
 Row(Country='Myanmar', Total_sale_by_country=58838467.85),
 Row(Country='South Korea', Total_sale_by_country=57434355.42999999),
 Row(Country='Ghana', Total_sale_by_country=56271382.65000001),
 Row(Country='Niger', Total_sale_by_country=55298211.27999998),
 Row(Country='Grenada', Total_sale_by_country=54988184.55000001),
 Row(Country='Republic of the Congo', Total_sale_by_country=54379808.38999998),
 Row(Country='Kosovo', Total_sale_by_country=53833142.79000001),
 Row(Country='Czech Republic', Total_sale_by_country=53543932.14),
 Row(Country='Ukraine', Total_sale_by_country=53252317.54),
 Row(Country='Vanuatu', Total_sale_by_country=51291723.21),
 Row(Country='Sudan', Total_sale_by_country=51237854.220000006),
 Row(Country='Mauritius', Total_sale_by_country=50891437.97),
 Row(Country='Bosnia and Herzegovina', Total_sale_by_country=50117508.48999999),
 Row(Country='Macedonia', Total_sale_by_country=49222085.24999999),
 Ro

In [17]:
df.groupby("Item Type").agg(f.sum("Units Sold")).show()

+---------------+---------------+
|      Item Type|sum(Units Sold)|
+---------------+---------------+
|      Baby Food|        2274921|
|         Cereal|        1949601|
|           Meat|        1901197|
|      Household|        2099238|
|     Vegetables|        2090330|
|      Beverages|        2208169|
|Office Supplies|        2107628|
|      Cosmetics|        2141909|
|  Personal Care|        2134895|
|         Fruits|        2341083|
|         Snacks|        1921075|
|        Clothes|        1983445|
+---------------+---------------+



In [18]:
Spark.sql("SELECT Country,sum(`Total Revenue`) as Total_sale_by_country FROM Ecommerce group by Country order by Total_sale_by_country desc").show()

+--------------------+---------------------+
|             Country|Total_sale_by_country|
+--------------------+---------------------+
|              Rwanda|  6.039873958999999E7|
|             Myanmar|        5.883846785E7|
|         South Korea|  5.743435542999999E7|
|               Ghana|  5.627138265000001E7|
|               Niger|  5.529821127999998E7|
|             Grenada|  5.498818455000001E7|
|Republic of the C...|  5.437980838999998E7|
|              Kosovo|  5.383314279000001E7|
|      Czech Republic|        5.354393214E7|
|             Ukraine|        5.325231754E7|
|             Vanuatu|        5.129172321E7|
|               Sudan| 5.1237854220000006E7|
|           Mauritius|        5.089143797E7|
|Bosnia and Herzeg...|  5.011750848999999E7|
|           Macedonia|  4.922208524999999E7|
|               Samoa| 4.7983045160000004E7|
|   Equatorial Guinea|  4.797403441000001E7|
|          San Marino|        4.788370848E7|
|             Andorra|  4.775669317000002E7|
|         

In [19]:
Spark.sql("""
SELECT e.Region, e.Country, e.`Sales Channel`, e.`Units Sold`
FROM Ecommerce e
JOIN (
    SELECT Region, MAX(`Units Sold`) as `Highest_Units_Sold`
    FROM Ecommerce
    GROUP BY Region
) max_units
ON e.Region = max_units.Region AND e.`Units Sold` = max_units.`Highest_Units_Sold`
ORDER BY e.`Units Sold` DESC
""").show()

+--------------------+-----------------+-------------+----------+
|              Region|          Country|Sales Channel|Units Sold|
+--------------------+-----------------+-------------+----------+
|              Europe|   United Kingdom|      Offline|      9999|
|Central America a...|         Dominica|       Online|      9996|
|                Asia|         Thailand|       Online|      9991|
|Australia and Oce...|            Samoa|       Online|      9990|
|  Sub-Saharan Africa|Equatorial Guinea|       Online|      9984|
|       North America|        Greenland|      Offline|      9898|
|Middle East and N...|            Libya|       Online|      9881|
+--------------------+-----------------+-------------+----------+



In [20]:
# This query first finds the maximum units sold for each region and then joins this result back to the original table to get the details of the country and sales channel with the highest units sold in each region. Let me know if this helps or if you need any further adjustments!